In [1]:
import run
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T


import numpy as np

from track import Track
from environment import Car, Environment
from sys import argv
import os
import sys

In [48]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))


# track = Track.read_from(argv[1])
# env = Environment(track)
tracks_path = ['../tracks/100.track','../tracks/101.track','../tracks/102.track']

tracks = [Track.read_from(x) for x in tracks_path]
envs = [Environment(x) for x in tracks]

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.l1 = nn.Linear(4, 32)
#         self.b1 = nn.BatchNorm1d(32)
        self.l2 = nn.Linear(32,16)
#         self.b2 = nn.BatchNorm1d(16)
        self.l3 = nn.Linear(16,64)
    
        self.l4 = nn.Linear(64,4)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu((self.l1(x)))
        x = F.relu((self.l2(x)))
        x = F.relu((self.l3(x)))
        x = F.relu(self.l4(x))
        return x

class DQN1(nn.Module):
    def __init__(self):
        super(DQN1, self).__init__()
        self.l1 = nn.Linear(4, 4)


    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.relu((self.l1(x)))
        return x
    
def get_input(i):
    return envs[i].state()


def send_action(i):
    broken, done, progress = envs[i].step(action)
    return (broken, done, progress)

BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.95
EPS_END = 0.2
EPS_DECAY = 200
TARGET_UPDATE = 100

# Get screen size so that we can initialize layers correctly based on shape
# returned from AI gym. Typical dimensions at this point are close to 3x40x90
# which is the result of a clamped and down-scaled render buffer in get_screen()
# init_screen = get_screen()
# _, _, screen_height, screen_width = init_screen.shape

# Get number of actions from gym action space
n_actions = 4

policy_net = DQN()
policy_net.eval()
target_net = DQN()
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
#    print(eps_threshold)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]],  dtype=torch.long)

def val_select_option(state):
    return policy_net(state).max(1)[1].view(1, 1)

episode_durations = []
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)),  dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
#     sys.stdout.write(f'\r Loss= {loss} \n val_proc= {val_proc}')

    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [44]:
state_proc = lambda state: torch.tensor(torch.from_numpy(np.array(state)).unsqueeze(0), dtype=torch.float32)
num_episodes = 500
# last_val = 0
# val_proc = 0
ex = True
while ex:
    for track in tracks:
        env = Environment(track)
        for i_episode in range(num_episodes):
            # Initialize the environment and state
            current_screen = state_proc(env.state())
            state = current_screen
            for t in count():

                # Select and perform an action
                action = select_action(state)
                broken, done, progress = env.step(action.item())
                reward = torch.tensor([ 100*progress + done - 1000*broken], dtype=torch.float32)

                # Observe new state
                current_screen = state_proc(env.state())
                if not done and not broken:
                    next_state = current_screen
                else:
                    next_state = None

                # Store the transition in memory
                if (next_state is not None):
                    memory.push(state, action, next_state, reward)

                # Move to the next state
                state = next_state

                # Perform one step of the optimization (on the target network)
                optimize_model()
                if done or broken:
                    episode_durations.append(t + 1)
                    break
            # Update the target network, copying all weights and biases in DQN
            if i_episode % TARGET_UPDATE == 0:
                val_proc = 0
                for j in range(3):
                    env = Environment(tracks[j])
                    vbroken = False
                    vdone = False
                    vcurrent_screen = state_proc(env.state())
                    vstate = vcurrent_screen 
                    while not vbroken and not vdone:
                        vaction = val_select_option(vstate)
                        vbroken, vdone, vprogress = env.step(vaction.item())
                        vstate = state_proc(env.state())
                    val_proc += vprogress
#                     print(f'Track={j} Progress={vprogress}')
                if (val_proc >= last_val):
                    last_val = val_proc
                    torch.save(policy_net.state_dict(),'weights.ckp')
                target_net.load_state_dict(policy_net.state_dict())
                sys.stdout.write(f'\r val_proc= {val_proc}')
                if val_proc >= 2.98:
                    ex = False
                    tracks = None
                    break


            

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


 val_proc= 3.051996151996152375

TypeError: 'NoneType' object is not subscriptable

In [52]:
env = Environment(tracks[1])
broken = False
done = False
last_screen = state_proc(env.state())
current_screen = state_proc(env.state())
state = current_screen - last_screen
while not broken and not done:
    action = val_select_option(state)
    broken, done, progress = env.step(action.item())
    state = state_proc(env.state())
print(progress)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.


1.0


In [33]:
torch.save(policy_net.state_dict(),'weights.ckp')

In [49]:
target_net.load_state_dict(torch.load('weights.ckp'))
policy_net = target_net

In [46]:
last_val

3.0

In [56]:
target_net.state_dict().keys()

odict_keys(['l1.weight', 'l1.bias', 'l2.weight', 'l2.bias', 'l3.weight', 'l3.bias', 'l4.weight', 'l4.bias'])

In [57]:
target_net.state_dict()['l1.weight']

tensor([[ 6.6044e-02, -1.2655e-01,  8.3839e-02, -3.6144e-01],
        [-2.2126e-01,  1.9400e-01, -3.3879e-01, -2.4960e-01],
        [-2.1212e+00,  1.3124e+00,  3.0072e-01, -1.8002e-01],
        [-3.4312e-01, -1.8527e-01,  1.9212e-03,  1.5076e-01],
        [-9.0965e+00, -3.8424e+00,  1.8285e+00,  5.1015e+00],
        [ 5.4418e-02, -2.8440e-01, -4.2868e-01, -1.1253e-01],
        [-1.9040e-01, -3.9458e-01, -2.1769e-01,  2.3344e-01],
        [ 1.4085e+00,  7.0795e-02, -1.9816e+00, -1.9705e+00],
        [-1.3508e+00,  8.1569e-01, -4.0656e+00, -4.3442e+00],
        [ 6.4903e-02, -2.9518e-01, -1.8339e-01,  5.2327e-02],
        [ 1.1716e-02, -5.9255e-02, -2.6268e-01,  1.0085e+00],
        [-2.7598e+00,  7.4275e-01,  9.6492e-01,  1.4070e+00],
        [-4.0847e-02,  4.0975e-01, -2.5788e-01,  2.3429e+00],
        [-2.4396e-01, -1.0527e-01,  3.8672e-01, -7.9331e-02],
        [-2.0522e-01,  1.1483e+00,  1.6004e+00,  1.9836e-01],
        [-8.2064e-01, -6.4770e+00,  9.6732e-01, -6.1755e-01],
        

In [59]:
wights = {'l1.weights' : [[ 6.6044e-02, -1.2655e-01,  8.3839e-02, -3.6144e-01],
        [-2.2126e-01,  1.9400e-01, -3.3879e-01, -2.4960e-01],
        [-2.1212e+00,  1.3124e+00,  3.0072e-01, -1.8002e-01],
        [-3.4312e-01, -1.8527e-01,  1.9212e-03,  1.5076e-01],
        [-9.0965e+00, -3.8424e+00,  1.8285e+00,  5.1015e+00],
        [ 5.4418e-02, -2.8440e-01, -4.2868e-01, -1.1253e-01],
        [-1.9040e-01, -3.9458e-01, -2.1769e-01,  2.3344e-01],
        [ 1.4085e+00,  7.0795e-02, -1.9816e+00, -1.9705e+00],
        [-1.3508e+00,  8.1569e-01, -4.0656e+00, -4.3442e+00],
        [ 6.4903e-02, -2.9518e-01, -1.8339e-01,  5.2327e-02],
        [ 1.1716e-02, -5.9255e-02, -2.6268e-01,  1.0085e+00],
        [-2.7598e+00,  7.4275e-01,  9.6492e-01,  1.4070e+00],
        [-4.0847e-02,  4.0975e-01, -2.5788e-01,  2.3429e+00],
        [-2.4396e-01, -1.0527e-01,  3.8672e-01, -7.9331e-02],
        [-2.0522e-01,  1.1483e+00,  1.6004e+00,  1.9836e-01],
        [-8.2064e-01, -6.4770e+00,  9.6732e-01, -6.1755e-01],
        [ 1.1569e-01, -3.6723e-01, -4.2259e-01, -3.6636e-01],
        [-6.7029e-02,  3.6742e-01, -5.9371e-02, -3.7632e-01],
        [ 9.0876e-03, -4.3256e-01, -1.1745e+00,  1.4715e+00],
        [-4.7834e-01, -9.8357e-02, -4.3519e-01, -1.6569e-01],
        [-1.5191e+00,  1.3942e-01,  3.7322e-01, -2.6922e-01],
        [-5.6686e-01, -1.7269e-01, -2.9061e-03, -3.9251e-01],
        [-4.6816e+00,  1.4266e+00, -6.6493e-01, -3.5835e+00],
        [-4.6864e-01, -5.4291e-01,  4.8899e-02, -3.8984e-01],
        [ 5.1053e-02, -6.3203e-01,  7.3636e-02, -2.3165e-01],
        [ 2.0914e+00, -1.2764e+00,  1.9427e-02,  2.3857e+00],
        [-1.1277e+00,  2.0157e+00, -3.8430e-01, -4.4020e+00],
        [ 6.8257e-01, -4.8308e+00, -2.3509e+00, -5.6325e-01],
        [ 3.5775e-01,  1.3983e-01,  6.0575e-01,  4.9504e-01],
        [-4.9670e+00,  6.0227e-01,  1.0295e+00, -6.1191e+00],
        [-1.3052e-01, -1.3927e-01, -3.2065e+00,  1.0153e-01],
        [-2.7553e+00,  1.8625e+00, -5.3413e-01, -6.4080e-01]]
         }

In [61]:
np.dot(wights['l1.weights'], [0,1,0,0])

array([-0.12655 ,  0.194   ,  1.3124  , -0.18527 , -3.8424  , -0.2844  ,
       -0.39458 ,  0.070795,  0.81569 , -0.29518 , -0.059255,  0.74275 ,
        0.40975 , -0.10527 ,  1.1483  , -6.477   , -0.36723 ,  0.36742 ,
       -0.43256 , -0.098357,  0.13942 , -0.17269 ,  1.4266  , -0.54291 ,
       -0.63203 , -1.2764  ,  2.0157  , -4.8308  ,  0.13983 ,  0.60227 ,
       -0.13927 ,  1.8625  ])